In [1]:
%%writefile app.py
from PIL import Image, ImageOps
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, Flatten, Dense, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Sequential
import streamlit as st
from tensorflow.keras.applications import ConvNeXtTiny, ResNet50 
from tensorflow.keras.applications.convnext import preprocess_input, decode_predictions
import plotly.express as px
import pandas as pd
from tensorflow.keras.utils import custom_object_scope

# Afficher la bannière
banniere_path = 'C:/Users/Lenovo/Documents/OC_IML_P7/presentation/Img/banniere.png'
image_ban = Image.open(banniere_path)
st.image(image_ban, caption='Bienvenue', use_column_width=True)

# Interface Streamlit
st.title("Classification d'images avec ConvNeXt")
st.header("Contexte")
st.write("Il s'agit ici de blablabla")

st.header("Les ressources :")
st.write("Pour nos travaux nous nous basons sur le dataset fourni par l'université de Standford.")
st.write("Vous trouverez ci dessous la répartition des images par race dans ce dataset :")
df_races = pd.read_csv('races.csv')
# Créer un graphique interactif avec Plotly
fig_races = px.bar(df_races, x="index", y="num_pictures", title="Répartition des photos par race")
# Afficher le graphique dans Streamlit
st.plotly_chart(fig_races)

st.header("Les résultats :")
st.write("Dans le cadre de notre étude nous avons souhaité tester la performance de l'algorithme ConvNeXt par rapport à ResNet.")
st.write("Nous avons réalisé nos tests pour différentes cardinalités (entre 5 et 30) de classes :")
df_results = pd.read_csv('results_vf.csv')
# Créer un graphique interactif avec Plotly
fig_results = px.line(df_results, x="Nb_races", y="Accuracy",color = "Model", title="Accuracy par algorithme")
# Afficher le graphique dans Streamlit
st.plotly_chart(fig_results)

# Créer un graphique interactif avec Plotly
fig_time = px.line(df_results, x="Nb_races", y="Temps",color = "Model", title="Temps par algorithme")
# Afficher le graphique dans Streamlit
st.plotly_chart(fig_time)

st.header("Testons notre algorithme !")
st.write("Téléchargez une image à classifier")

# Télécharger une image via Streamlit
uploaded_file = st.file_uploader("Choisissez une image...", type=["jpg", "jpeg", "png"])
list_races = ['Lévrier Afghan', 'Bouledogue français', 'Bichon maltais', 'Loulou de Poméranie', 'Deerhound']

#Définir la couche LayerScale
# https://github.com/andreped/keras/blob/810fd3c203dbae8bb1f888ebc91cdd31ff30cf34/keras/applications/convnext.py#L221
class LayerScale(layers.Layer):
    """Layer scale module.

    References:
      - https://arxiv.org/abs/2103.17239

    Args:
      init_values (float): Initial value for layer scale. Should be within
        [0, 1].
      projection_dim (int): Projection dimensionality.

    Returns:
      Tensor multiplied to the scale.
    """

    def __init__(self, init_values, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.init_values = init_values
        self.projection_dim = projection_dim

    def build(self, input_shape):
        self.gamma = tf.Variable(
            self.init_values * tf.ones((self.projection_dim,))
        )

    def call(self, x):
        return x * self.gamma

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "init_values": self.init_values,
                "projection_dim": self.projection_dim,
            }
        )
        return config

class StochasticDepth(layers.Layer):
    """Stochastic Depth module.

    It performs batch-wise dropping rather than sample-wise. In libraries like
    `timm`, it's similar to `DropPath` layers that drops residual paths
    sample-wise.

    References:
      - https://github.com/rwightman/pytorch-image-models

    Args:
      drop_path_rate (float): Probability of dropping paths. Should be within
        [0, 1].

    Returns:
      Tensor either with the residual path dropped or kept.
    """

    def __init__(self, drop_path_rate, **kwargs):
        super().__init__(**kwargs)
        self.drop_path_rate = drop_path_rate

    def call(self, x, training=None):
        if training:
            keep_prob = 1 - self.drop_path_rate
            shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x

    def get_config(self):
        config = super().get_config()
        config.update({"drop_path_rate": self.drop_path_rate})
        return config

# Utiliser custom_object_scope pour enregistrer LayerScale
@st.cache_resource
#def load_model(model_path):
#    with custom_object_scope({'LayerScale': LayerScale, "StochasticDepth": StochasticDepth}):
#        model = keras.models.load_model(model_path)
#    return model

def load_model(model_path):
    model = keras.models.load_model(model_path)
    return model


# Fonction de prédiction
def predict(model, img):
    # Redimensionner l'image à la taille attendue (224, 224 pour ConvNextTiny)
    img = img.resize((224, 224))

    # Convertir l'image en tableau numpy
    img_array = np.array(img)

    # Ajouter une dimension supplémentaire pour correspondre à l'entrée du modèle
    img_array = np.expand_dims(img_array, axis=0)

    # Prétraiter l'image pour le modèle ConvNextTiny
    img_array = preprocess_input(img_array)

    # Faire la prédiction
    prediction = model.predict(img_array)
    # Convert predictions to class labels
    predicted_class = int(np.argmax(prediction, axis=-1))
    
    # Invert the predicted class indices to actual labels
    labeled_prediction = list_races[predicted_class]
    
    return labeled_prediction

# Charger le modèle
model = load_model('C:/Users/Lenovo/Documents/OC_IML_P7/ConvNeXt_5.h5')

if uploaded_file is not None:
    # Afficher l'image téléchargée
    image = Image.open(uploaded_file)
    st.image(image, caption='Image téléchargée', use_column_width=True)

    # Prédire la classe de l'image
    st.write("Classification en cours...")
    labeled_prediction = predict(model, image)
    
    # Afficher les résultats
    st.write("Voici le résultat de la classification :")
    st.write(f"La race est: {labeled_prediction}")

Overwriting app.py
